In [28]:
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sns
import numpy as np

In [29]:
df = pd.read_csv('sierad.csv')
df.head()

,Room_T,RH,Teff,Wind_Speed
0,35,30,35.0,0.0
1,35,30,34.3,0.1
2,35,30,33.0,0.3
3,35,30,31.6,0.5
4,35,30,29.8,0.7


In [30]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [31]:
df.dtypes

Room_T          int64
RH              int64
Teff          float64
Wind_Speed    float64
dtype: object

In [32]:
data = spark.createDataFrame(df)

In [33]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [34]:
data.show(5)

+------+---+----+----------+
|Room_T| RH|Teff|Wind_Speed|
+------+---+----+----------+
|    35| 30|35.0|       0.0|
|    35| 30|34.3|       0.1|
|    35| 30|33.0|       0.3|
|    35| 30|31.6|       0.5|
|    35| 30|29.8|       0.7|
+------+---+----+----------+
only showing top 5 rows



In [41]:
from pyspark.ml.feature import VectorAssembler
vac = VectorAssembler(inputCols=["Room_T", "RH", "Teff"], outputCol="features")
va = vac.transform(data)

In [42]:
va.show(5)

+------+---+----+----------+----------------+
|Room_T| RH|Teff|Wind_Speed|        features|
+------+---+----+----------+----------------+
|    35| 30|35.0|       0.0|[35.0,30.0,35.0]|
|    35| 30|34.3|       0.1|[35.0,30.0,34.3]|
|    35| 30|33.0|       0.3|[35.0,30.0,33.0]|
|    35| 30|31.6|       0.5|[35.0,30.0,31.6]|
|    35| 30|29.8|       0.7|[35.0,30.0,29.8]|
+------+---+----+----------+----------------+
only showing top 5 rows



In [43]:
est = RandomForestRegressor()
{param[0].name: param[1] for param in est.extractParamMap().items()}

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featureSubsetStrategy': 'auto',
 'featuresCol': 'features',
 'impurity': 'variance',
 'labelCol': 'label',
 'maxBins': 32,
 'maxDepth': 5,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'numTrees': 20,
 'predictionCol': 'prediction',
 'seed': -5851613654371098793,
 'subsamplingRate': 1.0}

In [180]:
rf = RandomForestRegressor(numTrees=500, maxDepth=25, maxBins=128, seed=32, featuresCol='features', labelCol='Wind_Speed')

In [181]:
pipeline = Pipeline(stages=[vac, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

In [182]:
predictions.show()

+------+---+----+----------+----------------+------------------+
|Room_T| RH|Teff|Wind_Speed|        features|        prediction|
+------+---+----+----------+----------------+------------------+
|    21| 30|18.3|       2.5|[21.0,30.0,18.3]|1.8030812281670434|
|    21| 30|18.6|       1.9|[21.0,30.0,18.6]|1.6974585102516904|
|    21| 30|18.8|       1.5|[21.0,30.0,18.8]|1.4819465035894734|
|    21| 30|20.0|       1.1|[21.0,30.0,20.0]|1.0895024890780478|
|    21| 30|20.5|       0.9|[21.0,30.0,20.5]|0.8928977096517755|
|    21| 30|21.1|       0.7|[21.0,30.0,21.1]|0.7654707540820785|
|    21| 30|21.6|       0.5|[21.0,30.0,21.6]|0.6400669067191992|
|    21| 35|20.5|       0.7|[21.0,35.0,20.5]|0.7898549632491382|
|    21| 35|23.6|       0.0|[21.0,35.0,23.6]|0.3117705898444725|
|    21| 40|17.2|       2.5|[21.0,40.0,17.2]|1.9544696522281688|
|    21| 40|18.0|       1.7|[21.0,40.0,18.0]|1.7579526560165313|
|    21| 40|20.3|       0.5|[21.0,40.0,20.3]|0.8824438880470683|
|    21| 40|22.8|       0

In [109]:
# Select example rows to display.
predictions.select("prediction", "Wind_Speed", "features").show(5)


+------------------+----------+----------------+
|        prediction|Wind_Speed|        features|
+------------------+----------+----------------+
|               2.0|       2.5|[21.0,30.0,18.3]|
|1.8333333333333333|       1.9|[21.0,30.0,18.6]|
|1.8333333333333333|       1.5|[21.0,30.0,18.8]|
|1.2450757575757576|       1.1|[21.0,30.0,20.0]|
|0.9607620320855617|       0.9|[21.0,30.0,20.5]|
+------------------+----------+----------------+
only showing top 5 rows



In [110]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Wind_Speed", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

Root Mean Squared Error (RMSE) on test data = 0.295401
RandomForestRegressionModel (uid=RandomForestRegressor_45e0973ae2c072eb1e59) with 3 trees


In [187]:
predictionspd = predictions.toPandas()

In [189]:
predictionspd.shape

(961, 6)

In [190]:
from sklearn.metrics import r2_score

In [192]:
r2_score(predictionspd.Wind_Speed.values, predictionspd.prediction.values)

0.9105423601390011

In [188]:
predictionspd.select(predictions("prediction")).first().getList[Double](0)

TypeError: 'DataFrame' object is not callable

In [193]:
predictionspd.head()

,Room_T,RH,Teff,Wind_Speed,features,prediction
0,21,30,18.3,2.5,"[21.0, 30.0, 18.3]",1.803081
1,21,30,18.6,1.9,"[21.0, 30.0, 18.6]",1.697459
2,21,30,18.8,1.5,"[21.0, 30.0, 18.8]",1.481947
3,21,30,20.0,1.1,"[21.0, 30.0, 20.0]",1.089502
4,21,30,20.5,0.9,"[21.0, 30.0, 20.5]",0.892898


In [194]:
predictionspd.to_pickle("./sierad.pkl")

In [196]:
df = pd.read_pickle('sierad.pkl')

In [197]:
df

,Room_T,RH,Teff,Wind_Speed,features,prediction
0,21,30,18.3,2.5,"[21.0, 30.0, 18.3]",1.803081
1,21,30,18.6,1.9,"[21.0, 30.0, 18.6]",1.697459
2,21,30,18.8,1.5,"[21.0, 30.0, 18.8]",1.481947
3,21,30,20.0,1.1,"[21.0, 30.0, 20.0]",1.089502
4,21,30,20.5,0.9,"[21.0, 30.0, 20.5]",0.892898
5,21,30,21.1,0.7,"[21.0, 30.0, 21.1]",0.765471
6,21,30,21.6,0.5,"[21.0, 30.0, 21.6]",0.640067
7,21,35,20.5,0.7,"[21.0, 35.0, 20.5]",0.789855
8,21,35,23.6,0.0,"[21.0, 35.0, 23.6]",0.311771
9,21,40,17.2,2.5,"[21.0, 40.0, 17.2]",1.954470
